# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [47]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [48]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-10-events.csv', '/workspace/home/event_data/2018-11-20-events.csv', '/workspace/home/event_data/2018-11-09-events.csv', '/workspace/home/event_data/2018-11-11-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-27-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-24-events.csv', '/workspace/home/event_data/2018-11-23-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-30-events.csv', '/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-01-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-13-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [49]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_data_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [50]:
# check the number of rows in your csv file
with open('event_data_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [53]:
# check the data
df_event_data_new = pd.read_csv('event_data_new.csv')
df_event_data_new.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Hoobastank,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",132,Say The Same,96
1,Mark Knopfler,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",132,Why Aye Man,96
2,Mogwai,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",132,We're No Here,96
3,The Casualties,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",132,Punx Unite,96
4,The Living End,Ryan,M,0,Smith,188.62975,free,"San Jose-Sunnyvale-Santa Clara, CA",433,Roll On (Album Version),26


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_data_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_data_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [54]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [55]:
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication =
    {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [56]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [57]:
# Dropping the table just incase we want to fix the table

# query = "drop table event_data_new_1"
# try:
#     session.execute(query)
# except Exception as e:
#     print (e)


# Creating table event_datafile_new_1
# The primary key is a combination between session_id and item_in_session because we want to filter using those keys
# Remember, with Apache Cassandra you model the database tables on the queries you want to run.
query = "create table if not exists event_data_new_1"
query = query + """
    (artist text, first_name text,
    gender text, item_in_session varchar,
    last_name text, length varchar, level text, 
    location text, session_id varchar, song text, user_id varchar, 
    primary key(session_id, item_in_session))
    """
try:
    session.execute(query)
except Exception as e:
    print (e)

In [58]:


# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_data_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into event_data_new_1 (\
            artist, first_name,\
            gender, item_in_session,\
            last_name, length,\
            level, location,\
            session_id, song, \
            user_id\
            )"
        query = query + "values (\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s) "
        session.execute(query, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10] ))

#### Do a SELECT to verify that the data have been inserted into each table

In [60]:
query = """ 
    select 
      *
    from event_data_new_1
    limit 4 """
try:
    rows = session.execute (query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.first_name, row.gender, row.item_in_session, row.last_name,
           row.length, row.level, row.location, row.session_id, row.song, row.user_id)

Pavement Kynnedi F 0 Sanchez 99.16036 free Cedar Rapids, IA 631 Mercy:The Laundromat 89
Parachute Dominick M 0 Norris 146.05016 free Los Angeles-Long Beach-Anaheim, CA 403 She Is Love 45
Hoobastank Noah M 1 Chavez 232.17587 free Ogden-Clearfield, UT 492 Born To Lead 94
Kanye West Lily F 1 Burns 238.52363 free New York-Newark-Jersey City, NY-NJ-PA 517 Flashing Lights 32


In [61]:
## Query 1:  Give me the artist, song title and song's length in the music app
## history that was heard during \
## sessionId = 338, and itemInSession = 4


##  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = """ 
    select 
        artist,
        song,
        length 
    from event_data_new_1
    where session_id = '338'
        and item_in_session = '4' """
try:
    rows = session.execute (query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)
                    

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Question 2

In [62]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


# Dropping the table just incase we want to fix the table
# query = "drop table event_data_new_2"
# try:
#     session.execute(query)
# except Exception as e:
#     print (e)


# Creating table event_data_new
# Creating table event_data_new_2
# The primary key is a combination between user_id and session_id because we want to filter using those keys
# Remember, with Apache Cassandra you model the database tables on the queries you want to run.

query = "create table if not exists event_data_new_2"
query = query + """
    (artist text, first_name text,
    gender text, item_in_session varchar,
    last_name text, length varchar, level text, 
    location text, session_id varchar, song text, user_id varchar, 
    primary key(user_id, session_id))
    """
try:
    session.execute(query)
except Exception as e:
    print (e)



# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_data_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into event_data_new_2 (\
            artist, first_name,\
            gender, item_in_session,\
            last_name, length,\
            level, location,\
            session_id, song, \
            user_id\
            )"
        query = query + "values (\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s) "
        session.execute(query, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10] ))

        


In [63]:
query = """ 
    select 
        artist,
        song,
        first_name,
        last_name
    from event_data_new_2
    where user_id = '10'
        and session_id = '182' """
try:
    rows = session.execute (query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)
                    

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Question 3

In [64]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Dropping the table just incase we want to fix the table
# query = "drop table event_data_new_3"
# try:
#     session.execute(query)
# except Exception as e:
#     print (e)


# Creating table event_data_new
# Creating table event_data_new_3
# The primary key is song because we want to filter that key
# Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Creating table event_data_new_3
query = "create table if not exists event_data_new_3"
query = query + """
    (artist text, first_name text,
    gender text, item_in_session varchar,
    last_name text, length varchar, level text, 
    location text, session_id varchar, song text, user_id varchar, 
    primary key(song))
    """
try:
    session.execute(query)
except Exception as e:
    print (e)



# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_data_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into event_data_new_3 (\
            artist, first_name,\
            gender, item_in_session,\
            last_name, length,\
            level, location,\
            session_id, song, \
            user_id\
            )"
        query = query + "values (\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s, %s,\
            %s) "
        session.execute(query, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10] ))

        


In [65]:
query = """ 
    select 
        first_name,
        last_name
    from event_data_new_3
    where song = 'All Hands Against His Own' """
try:
    rows = session.execute (query)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)
                    

Jacqueline Lynch


### Drop the tables before closing out the sessions

In [66]:
query = "DROP table event_data_new_1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP table event_data_new_2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP table event_data_new_3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [67]:
session.shutdown()
cluster.shutdown()